In [68]:
import random
import numpy as np
import math

In [85]:
adjacency_list1 = [[1,2,5], [0,5,2],[0,1,3,5], [2,4], [3,5],[0,1,2,4]]
vertics_cost1 = [8,4,1,6,5,4]

vertics_cost = [1,2,3,4,5,6,6,5,4,1,2,3,3,2,1,2,5,4,3,2,4,3,2,4,2,1,3,1,1,1,3,1]
adjacency_list = [[1,3],[0,3,20,11,6,28],[9,10,24],[0,1,9,12,4,5],[3,19,5],[3,4], [1,7,8], [6,8,14],[6,7,9], [8,20,3,2,10,12,30,23,16,29,15],[2,17,9],[1,13,18,25,12],[11,3,9],[18,11],[7,15,27],[14,12],[22,23,17,9],[16,10],[19,11,27,13],[18,27,4,31,26],[1,9],[28,22],[29,21,16],[16,9,24,30],[23,31,2],[28,11,29],[19,31],[14,18,19],[21,1,25],[25,22,9],[23,9],[19,26,24]]

In [45]:
def remove_node(adjacency_list, node):
    graph = adjacency_list.copy()
    graph[node] = [-1]
    return [[ele for ele in sub if ele != node] for sub in graph]

In [66]:
# klasa graf, koja racuna broj komponenti grafa
class Graph:
    def __init__(self,adjacency_list, cost):
        self.cost = cost
        self.adj = adjacency_list
        self.V = len(self.adj)
        self.cost = cost
    def DFSUtil(self, temp, v, visited):
        visited[v] = True
        temp.append(v)
        for i in self.adj[v]:
            if visited[i] == False:
                temp = self.DFSUtil(temp, i, visited)
        return temp
    def connectedComponents(self):
        visited = []
        cc = []
        for i in range(self.V):
            if self.adj[i] != [] and self.adj[i][0] == -1:
                visited.append(True)
            else:
                visited.append(False)
        for v in range(self.V):
            if visited[v] == False:
                temp=[]
                cc.append(self.DFSUtil(temp, v, visited))
        return cc
    def remove_node(self,node):
        newAdj = self.adj.copy()
        newAdj[node] = [-1]
        newAdj = [[ele for ele in sub if ele != node] for sub in newAdj]
        return Graph(newAdj, self.cost)
    def maxCost(self):
        maks = -float('inf')
        for i in range(self.V):
            if (self.adj[i] != [] and self.adj[i][0] != -1) or self.adj[i] == []:
                if self.cost[i] > maks:
                    maks = self.cost[i]
        return maks

graph = Graph(adjacency_list, vertics_cost)
graph1 = graph.remove_node(4)
graph1 = graph1.remove_node(0)
print(graph1.adj)
cc = graph.connectedComponents()
print(cc)
print(len(cc))
print(graph.maxCost())
print(graph1.maxCost())
print(len(graph1.connectedComponents()))


[[-1], [], [3], [2], [-1]]
[[0, 1, 4, 3, 2]]
1
9
7
2


In [86]:
#klasa sadrzi informacije o genomu 
class Individual:
    def __init__(self, V, adjacency_list, cost):
        self.graph = Graph(adjacency_list, cost)
        self.code = []
        for i in range(V):
            self.code.append(random.random() < 0.5)
        self.fitness = self.fitnessFunction()
        self.subgraph1, self.subgraph2 = self.getSubgraphs()
    def __lt__(self, other):
        return self.fitness < other.fitness
    def fitnessFunction(self):
        w1 = 0
        w2 = 0
        graph1 = Graph(self.graph.adj, self.graph.cost)
        graph2 = Graph(self.graph.adj, self.graph.cost)
        for i in range(len(self.code)):
            if self.code[i]:
                w1 += vertics_cost[i]
                graph2 = graph2.remove_node(i)
            else:
                w2 += vertics_cost[i]
                graph1 = graph1.remove_node(i)
                
        nc1 = len(graph1.connectedComponents())
        nc2 = len(graph2.connectedComponents())
        max1 = graph1.maxCost()
        max2 = graph2.maxCost()
        penalty_function = (nc1-1)*max1 + (nc2-1)*max2
        return abs(w1-w2)+penalty_function
    def getSubgraphs(self):
        sub1 = adjacency_list.copy()
        sub2 = adjacency_list.copy()
        for i in range(len(self.code)):
            if self.code[i]:
                sub2 = remove_node(sub2, i)
            else:
                sub1 = remove_node(sub1, i)
        return sub1, sub2

In [87]:
Npop = 100
probability = 0.05


In [88]:
def selection(population):
    min = float('inf')
    for i in range(6):
        j = random.randrange(Npop)
        if population[j].fitness < min:
            min = population[j].fitness
            k = j
    return k

In [89]:
def crossover(parent1, parent2, child1, child2):
    i = random.randrange(len(parent1.code))
    for j in range(i):
        child1.code[j] = parent1.code[j]
        child2.code[j] = parent2.code[j]
    for j in range(i, len(parent1.code)):
        child1.code[j] = parent2.code[j]
        child2.code[j] = parent1.code[j]
    

In [90]:
def mutation(individual):
    for i in range(len(individual.code)):
        if random.random() > probability:
            continue
        individual.code[i] = not individual.code[i]
        

In [91]:
#pocetni kod genetskog algoritma
#potrebno povecati vrednost fitnes funkcije ukoliko dobijeni grafovi nisu povezani
population = []
newPopulation = []
V = len(adjacency_list)
for i in range(Npop):
    population.append(Individual(V, adjacency_list, vertics_cost))
    newPopulation.append(Individual(V,  adjacency_list, vertics_cost))

for iteration in range(500):
    population.sort()
    for i in range(30):
        newPopulation[i] = population[i]
        
    for i in range(30,Npop,2):
        k1 = selection(population)
        k2 = selection(population)
        crossover(population[k1], population[k2],newPopulation[i],newPopulation[i+1])
        mutation(newPopulation[i])
        mutation(newPopulation[i+1])
        newPopulation[i].fitness = newPopulation[i].fitnessFunction()
        newPopulation[i+1].fitness = newPopulation[i+1].fitnessFunction()
    population = newPopulation
population.sort()
print('Solution ', population[0].fitness)
print('Graph 1', population[0].subgraph1)
print('Graph 2',population[0].subgraph2)
print('Vertics', population[0].code)


    

Solution  0
Graph 1 [[-1], [-1], [-1], [12, 4, 5], [3, 19, 5], [3, 4], [7], [6, 14], [-1], [-1], [-1], [13, 18, 12], [11, 3], [18, 11], [7, 15], [14, 12], [23, 17], [16], [19, 11, 13], [18, 4, 31, 26], [], [], [-1], [16, 30], [-1], [-1], [19, 31], [-1], [-1], [], [23], [19, 26]]
Graph 2 [[1], [0, 28], [9, 10, 24], [-1], [-1], [-1], [-1], [-1], [9], [8, 2, 10], [2, 9], [-1], [-1], [-1], [-1], [-1], [-1], [-1], [-1], [-1], [-1], [-1], [], [-1], [2], [28], [-1], [], [1, 25], [-1], [-1], [-1]]
Vertics [True, False, True, True, False, False, True, True, True, True, False, False, True, False, True, True, False, False, False, False, True, True, True, False, False, True, False, False, True, True, True, False]
